In [4]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
pd.pandas.set_option('display.max_columns',None)

In [35]:
train_procreesed_data = pd.read_csv("../data/train_processed_data.csv")
print(train_data.shape)

(162902, 2)


In [36]:
train_procreesed_data = train_procreesed_data.dropna()

In [37]:
train_procreesed_data.head()

,clean_text,category
0,modi promise minimum government maximum govern...,-1.0
1,talk nonsense continue drama vote modi,0.0
2,say vote modi welcome bjp tell rahul main camp...,1.0
3,ask supporter prefix chowkidar name modi great...,1.0
4,answer among powerful world leader today trump...,1.0


In [44]:
cat1 = train_procreesed_data[train_procreesed_data["category"] == -1].iloc[0:3550]
cat2 = train_procreesed_data[train_procreesed_data["category"] == 0].iloc[0:5515]
cat3 = train_procreesed_data[train_procreesed_data["category"] == 1].iloc[0:7224]

In [45]:
train_data = cat1.append([cat2, cat3],ignore_index = True)

In [46]:
train_data.shape

(16289, 2)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tfidfvector=TfidfVectorizer(ngram_range=(2,2))                                     
traindataset = tfidfvector.fit_transform(train_data['clean_text'])   

In [48]:
from sklearn.model_selection import train_test_split, GridSearchCV,  KFold
from sklearn.metrics import confusion_matrix                              
from sklearn.metrics import accuracy_score                                
from sklearn.metrics import roc_curve, auc                                

In [49]:
traindataset                                                              

<16289x143526 sparse matrix of type '<class 'numpy.float64'>'
	with 224128 stored elements in Compressed Sparse Row format>

In [50]:
x = traindataset
y = train_data['category'].values
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state=303)

In [51]:
from sklearn import metrics
def model_compare(model_name,model,y_test,y_pred,y_score, roc_auc = None):
    binclass_metrics = {
                        'Accuracy' : metrics.accuracy_score(y_test, y_pred),
                        'Precision' : metrics.precision_score(y_test, y_pred),
                        'Recall' : metrics.recall_score(y_test, y_pred),
                        'F1 Score' : metrics.f1_score(y_test, y_pred),
                        'ROC AUC' : roc_auc
                       }

    df_metrics = pd.DataFrame.from_dict(binclass_metrics, orient='index')
    df_metrics.columns = [model_name] 
    return df_metrics

## DecisionTree

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
model_dt = DecisionTreeClassifier(random_state=303)
np.random.seed(303)
# start = time.time()
param_dist = {'max_depth': [2, 3, 4, 5, 6, 7],
              'min_samples_leaf':[100,200,70, 30],
              'criterion': ['gini', 'entropy']}

cv_rf = GridSearchCV(model_dt, cv = 10,
                     param_grid=param_dist, 
                     n_jobs = 1)

cv_rf.fit(x_train, y_train)
print('Best Parameters using grid search: \n', cv_rf.best_params_)

Best Parameters using grid search: 
 {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 70}


In [54]:
model_dt = DecisionTreeClassifier(random_state=303,
                                  max_depth=7, 
                                  criterion = 'gini', 
                                  min_samples_leaf=70 )
model_dt.fit(x_train, y_train)
model_dt_score_train = model_dt.score(x_train, y_train)
print("Training score: ",model_dt_score_train)
model_dt_score_test = model_dt.score(x_test, y_test)
print("Testing score: ",model_dt_score_test)            

Training score:  0.4682679763640549
Testing score:  0.46838551258440764


In [55]:
y_pred = model_dt.predict(x_test)
# Model Accuracy
print (model_dt.score(x_test, y_test))

0.46838551258440764


In [24]:
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [57]:
y_pred = model_dt.predict(x_test)
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr_dt, tpr_dt)
y_score = accuracy_score(y_test, y_pred)
metrix_dt = model_compare("Decision Tree",model_dt, y_test, y_pred, y_score, roc_auc  )
metrix_dt

ValueError: multiclass format is not supported

## Random Forest

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
model_rf = RandomForestClassifier(random_state=303)

In [60]:
param_dist = {'max_depth': [3,4,5,7,8],
              'bootstrap': [True, False],
              'max_features': ['auto', 'sqrt', 'log2', None],
              'criterion': ['gini', 'entropy']}

cv_rf = GridSearchCV(model_rf, cv = 10,
                     param_grid=param_dist, 
                     n_jobs = 1)

cv_rf.fit(x_train, y_train)
print('Best Parameters using grid search: \n', cv_rf.best_params_)

Best Parameters using grid search: 
 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 8, 'max_features': None}


In [ ]:
model_rf.set_params(criterion    = 'gini',
                    max_features = 'auto', 
                    max_depth    = 8,
                    bootstrap    = True
                   )

In [22]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(x_train, y_train)

ValueError: Found array with 0 feature(s) (shape=(130384, 0)) while a minimum of 1 is required.

In [29]:
train_data.isnull().sum()

clean_text              0
category                0
clean_processed_text    0
dtype: int64